In [ ]:
# Based on Ch. 3 example of "Deep Learning with Python" by F. Chollet
# Adds k-fold validation and sklearn StandardScaler to the example

from keras.datasets import boston_housing
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# Shuffle for kicks
train_data, train_targets = shuffle(train_data, train_targets, random_state = 0)
test_data, test_targets = shuffle(test_data, test_targets, random_state = 0)

# Targets to 2D arrays
train_targets = np.reshape(train_targets, (train_targets.shape[0], 1))
test_targets = np.reshape(test_targets, (test_targets.shape[0], 1))

In [ ]:
# Scale features

from sklearn.preprocessing import StandardScaler

sc_features = StandardScaler()
X_train = sc_features.fit_transform(train_data)
X_test = sc_features.transform(test_data)

# Scaling for targets, skipped now
# sc_labels = StandardScaler()
# sc_labels = sc_labels.fit(np.reshape(train_targets, (train_targets.shape[0], 1)))
# Y_train = sc_labels.transform(train_targets)
# Y_test = sc_labels.transform(test_targets)

Y_train = train_targets
Y_test = test_targets

In [ ]:
from keras import models, layers
from sklearn.model_selection import KFold

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1], )))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1)) # 1D regression
    model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
    return model

def train_for(X_train, Y_train, X_validation, Y_validation, epochs = 250):
    model = build_model()
    history = model.fit(X_train, Y_train, epochs = epochs, verbose = 0, validation_data = (X_validation, Y_validation))
    return model, history

def k_fold_validate(folds):
    kf = KFold(n_splits=folds)
    split_indices = kf.split(X_train)
    histories = []
    for train_index, test_index in split_indices:
        X_train_split = X_train[train_index]
        Y_train_split = Y_train[train_index]
        X_test_split = X_train[test_index]
        Y_test_split = Y_train[test_index]
        model, history = train_for(X_train_split, Y_train_split, X_test_split, Y_test_split)
        histories.append(history)
    return histories
    
histories = k_fold_validate(5)


In [ ]:
# Plot training and validation mean absolute errors vs. training epoch

def average_for_quantity(histories, quantity):
    vals = [history.history[quantity] for history in histories]
    return np.mean(vals, axis = 0)

# train_losses = average_for_quantity(histories, 'loss')
# val_losses = average_for_quantity(histories, 'val_loss')

train_maes = average_for_quantity(histories, 'mean_absolute_error')
val_maes = average_for_quantity(histories, 'val_mean_absolute_error')

arg_val_min = np.argmin(val_maes)

epochs = range(0, len(train_maes))

plt.clf()
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_maes, 'b-', label='Training mean absolute error')
plt.plot(epochs, val_maes, 'r-', label='Validation mean absolute error')
plt.plot([arg_val_min, arg_val_min], [0, np.amax(val_maes)], 'g--', label = 'Validation minimum')
plt.title('Training and validation loss, min validation %.2f' % (val_maes[arg_val_min]))
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.xlim([10, len(epochs)])
plt.ylim([0, max(val_maes[10:])])
plt.legend()
plt.show()


In [ ]:
# Train model once more with all data and optimal number of epochs, without validation data
model, history = train_for(X_train, Y_train, np.zeros((0, X_train.shape[1])), np.zeros((0, Y_train.shape[1])), 
                           epochs = arg_val_min + 1)

evaluated = model.evaluate(X_test, Y_test)

# Dictionary of computed losses and metrics
test_results = dict(zip(model.metrics_names, evaluated))

print(test_results)

In [ ]:
# Another example utilizing EarlyStopping callback when validation losses stop improving
from keras.callbacks import EarlyStopping

earlyStop = EarlyStopping(monitor = 'val_mean_absolute_error', min_delta = 0, patience = 5, verbose = 1, mode = 'auto')

def train_with_early_stop(X_train, Y_train, X_validation, Y_validation, epochs = 250):
    model = build_model()
    history = model.fit(X_train, Y_train, epochs = epochs, verbose = 0, validation_data = (X_validation, Y_validation), 
                       callbacks = [earlyStop])
    return model

def k_fold_validate_early_stop(folds):
    kf = KFold(n_splits=folds)
    split_indices = kf.split(X_train)
    metrics = []
    for train_index, test_index in split_indices:
        X_train_split = X_train[train_index]
        Y_train_split = Y_train[train_index]
        X_test_split = X_train[test_index]
        Y_test_split = Y_train[test_index]
        model = train_with_early_stop(X_train_split, Y_train_split, X_test_split, Y_test_split)
        evaluated = model.evaluate(X_test_split, Y_test_split)
        metrics.append(dict(zip(model.metrics_names, evaluated)))
    return np.mean([metrics_for_fold['mean_absolute_error'] for metrics_for_fold in metrics])

mean_absolute_error = k_fold_validate_early_stop(folds = 5)
print('Mean absolute error: %.2f' % (mean_absolute_error))